In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def generate_sample(min_fc, max_fc, phase=0, amplitude=1, n=180, fs=4000):
  """
  Generates data which has a carrier frequency between min_fc and max_fc
  :param min_fc: minimum fc possible
  :param max_fc: maximum fc possible
  :param n: number of points to generate
  :param phase_offset: The desired phase offset in degrees
  :param fs: The sampling frequency in Hertz for calculating digital frequency
  :return: One set of data with fc randomly between min_fc and max_fc of size n, and the digital frequency
  """

  t = np.arange(0, n) / fs
  fc = np.random.uniform(min_fc, max_fc)
  digital_freq = fc / fs
  x = amplitude * np.exp(1j * (2 * np.pi * fc * t + np.deg2rad(phase)))

  return np.array(x), digital_freq

In [4]:
class Wave:
  def __init__(self):
    self.freq = []
    self.m = 0
    self.x = np.zeros(180, dtype=np.complex128)
  
  def append(self, sample):
    self.x += sample[0]
    self.freq.append(sample[1])

In [5]:
def complex_to_real(data):
    real_data = np.real(data)
    imag_data = np.imag(data)
    
    real_data = np.hstack((real_data, imag_data))
    
    real_data = real_data.astype(float)
    
    return real_data

In [6]:
# Generating Training Data Points 
waves_train = []
n = 10000
max_m = 6
X_train = []
y_train = []

for i in range(n):
  m = np.random.choice(range(1, max_m + 1))
  wave = Wave()
  wave.m = m
  for _ in range(m):
    wave.append(generate_sample(40, 600))
  waves_train.append(wave)
  X_train.append(complex_to_real(wave.x))
  y_train.append(wave.m)

X_train = np.array(X_train)
y_train = np.array(y_train, dtype=float)

In [7]:
# Generating Testing Data Points 
waves_test = []
n = 100
max_m = 6
X_test = []
y_test = []

for i in range(n):
  m = np.random.choice(range(1, max_m + 1))
  wave = Wave()
  wave.m = m
  for _ in range(m):
    wave.append(generate_sample(40, 600))
  waves_test.append(wave)
  X_test.append(complex_to_real(wave.x))
  y_test.append(wave.m)

X_test = np.array(X_test)
y_test = np.array(y_test, dtype=float)

In [11]:
import tensorflow as tf

def L_est(y_target, y_pred):
    cond = tf.less_equal(y_pred, y_target)
    return 100 * (tf.where(cond, tf.math.exp(y_target - y_pred) - 1, 0.5 * tf.math.square(y_target - y_pred)))

In [12]:
from keras.models import Sequential
from keras.layers import Dense

# define the model
model = Sequential()
model.add(Dense(150, input_dim=360, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(1, activation='relu'))

# compile the model
model.compile(loss=L_est, optimizer='adam', metrics=['accuracy'])

# fit the model to the training data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10000, batch_size=10)

# evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


Epoch 1/10000
1000/1000 [==============================] - 4s 3ms/step - loss: 323.7386 - accuracy: 0.1622 - val_loss: 4.7745 - val_accuracy: 0.1200
Epoch 2/10000
1000/1000 [==============================] - 2s 2ms/step - loss: 4.0628 - accuracy: 0.1680 - val_loss: 4.0812 - val_accuracy: 0.1200
Epoch 3/10000
1000/1000 [==============================] - 2s 2ms/step - loss: 3.6730 - accuracy: 0.1680 - val_loss: 2.2934 - val_accuracy: 0.1200
Epoch 4/10000
1000/1000 [==============================] - 4s 4ms/step - loss: 3.1693 - accuracy: 0.1680 - val_loss: 6.5240 - val_accuracy: 0.1200
Epoch 5/10000
1000/1000 [==============================] - 2s 2ms/step - loss: 3.0981 - accuracy: 0.1680 - val_loss: 2.6770 - val_accuracy: 0.1200
Epoch 6/10000
1000/1000 [==============================] - 3s 3ms/step - loss: 2.7856 - accuracy: 0.1680 - val_loss: 2.5358 - val_accuracy: 0.1200
Epoch 7/10000
1000/1000 [==============================] - 2s 2ms/step - loss: 2.3290 - accuracy: 0.1680 - val_loss:

KeyboardInterrupt: ignored

In [ ]:
_n = 235
print(waves[_n].m)
print(waves[_n].freq)
plt.scatter(waves[_n].x.real, waves[_n].x.imag)
plt.plot(waves[_n].x.real, waves[_n].x.imag)